In [2]:
import yfinance as yf
import pandas as pd

# Download data for EUR/USD and USD/JPY
eur_usd= yf.download('EURUSD=X', start='2012-01-01', end='2022-12-31')
jpy_usd=yf.download('JPYUSD=X', start='2012-01-01', end='2022-12-31')

# Save the Data as csv
eur_usd.to_csv('eur_usd.csv')
jpy_usd.to_csv('jpy_usd.csv')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [8]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Load the data
eur_usd = pd.read_csv('eur_usd.csv', index_col='Date', parse_dates=True)
jpy_usd = pd.read_csv('jpy_usd.csv', index_col='Date', parse_dates=True)

# Combine the datasets
data = pd.concat([eur_usd['Close'], jpy_usd['Close']], axis=1)
data.columns = ['EURUSD', 'USDJPY']

# Drop rows with missing values
data.dropna(inplace=True)

# Normalize the data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# Convert to a DataFrame
data_scaled = pd.DataFrame(data_scaled, columns=data.columns, index=data.index)

# Split the data into training and testing sets
train_size = int(len(data_scaled) * 0.8)
train_data = data_scaled[:train_size]
test_data = data_scaled[train_size:]

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class GRU_LSTM_DQN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GRU_LSTM_DQN, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True)
        self.lstm = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.gru(x)
        out, _ = self.lstm(out)
        out = self.fc(out[:, -1, :])
        return out

class GRU_DQN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GRU_DQN, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.gru(x)
        out = self.fc(out[:, -1, :])
        return out

class LSTM_DQN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTM_DQN, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

# Convert data to PyTorch tensors
train_tensor = torch.tensor(train_data.values, dtype=torch.float32)
test_tensor = torch.tensor(test_data.values, dtype=torch.float32)

# Create data loaders
train_loader = DataLoader(TensorDataset(train_tensor, train_tensor), batch_size=32, shuffle=True)
test_loader = DataLoader(TensorDataset(test_tensor, test_tensor), batch_size=32, shuffle=False)

# Initialize models, loss function, and optimizer
input_size = train_data.shape[1]
hidden_size = 50
output_size = 1

gru_lstm_dqn = GRU_LSTM_DQN(input_size, hidden_size, output_size)
gru_dqn = GRU_DQN(input_size, hidden_size, output_size)
lstm_dqn = LSTM_DQN(input_size, hidden_size, output_size)

criterion = nn.MSELoss()
gru_lstm_optimizer = optim.Adam(gru_lstm_dqn.parameters(), lr=0.001)
gru_optimizer = optim.Adam(gru_dqn.parameters(), lr=0.001)
lstm_optimizer = optim.Adam(lstm_dqn.parameters(), lr=0.001)

# Training function
def train_model(model, optimizer, train_loader, num_epochs=50):
    model.train()
    for epoch in range(num_epochs):
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs.unsqueeze(1))
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

# Train the models
train_model(gru_lstm_dqn, gru_lstm_optimizer, train_loader)
train_model(gru_dqn, gru_optimizer, train_loader)
train_model(lstm_dqn, lstm_optimizer, train_loader)


c:\Python312\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([32, 2])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Python312\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([20, 2])) that is different to the input size (torch.Size([20, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/50, Loss: 0.02591935358941555
Epoch 2/50, Loss: 0.02034868113696575
Epoch 3/50, Loss: 0.011470948345959187
Epoch 4/50, Loss: 0.006425137631595135
Epoch 5/50, Loss: 0.01271386444568634
Epoch 6/50, Loss: 0.01203925535082817
Epoch 7/50, Loss: 0.010944143868982792
Epoch 8/50, Loss: 0.017211580649018288
Epoch 9/50, Loss: 0.01495576836168766
Epoch 10/50, Loss: 0.009895658120512962
Epoch 11/50, Loss: 0.006396540440618992
Epoch 12/50, Loss: 0.011937248520553112
Epoch 13/50, Loss: 0.01562761329114437
Epoch 14/50, Loss: 0.007457879837602377
Epoch 15/50, Loss: 0.007917645387351513
Epoch 16/50, Loss: 0.0067226700484752655
Epoch 17/50, Loss: 0.011585848405957222
Epoch 18/50, Loss: 0.007037130184471607
Epoch 19/50, Loss: 0.012033526785671711
Epoch 20/50, Loss: 0.010453899390995502
Epoch 21/50, Loss: 0.016190819442272186
Epoch 22/50, Loss: 0.01156492531299591
Epoch 23/50, Loss: 0.022240495309233665
Epoch 24/50, Loss: 0.00286888494156301
Epoch 25/50, Loss: 0.007952190935611725
Epoch 26/50, Los

In [10]:
import numpy as np

# Function to calculate performance metrics
def evaluate_model(model, test_loader):
    model.eval()
    returns = []
    with torch.no_grad():
        for inputs, _ in test_loader:
            outputs = model(inputs.unsqueeze(1))
            returns.append(outputs.squeeze().numpy())

    returns = np.concatenate(returns)

    # Calculate Sharpe Ratio
    sharpe_ratio = np.mean(returns) / np.std(returns) * np.sqrt(252)

    # Calculate maximum drawdown
    cumulative_returns = np.cumprod(1 + returns) - 1
    drawdown = cumulative_returns - np.maximum.accumulate(cumulative_returns)
    max_drawdown = np.min(drawdown)

    # Calculate annual returns
    annual_returns = np.mean(returns) * 252

    return sharpe_ratio, max_drawdown, annual_returns

# Evaluate the models
gru_lstm_metrics = evaluate_model(gru_lstm_dqn, test_loader)
gru_metrics = evaluate_model(gru_dqn, test_loader)
lstm_metrics = evaluate_model(lstm_dqn, test_loader)

print('GRU-LSTM-DQN Metrics:', gru_lstm_metrics)
print('GRU-DQN Metrics:', gru_metrics)
print('LSTM-DQN Metrics:', lstm_metrics)


GRU-LSTM-DQN Metrics: (35.37224389473442, nan, 84.87436079978943)
GRU-DQN Metrics: (35.064564183752545, nan, 84.70393967628479)
LSTM-DQN Metrics: (35.7964682626438, nan, 85.11770582199097)


c:\Python312\Lib\site-packages\numpy\core\fromnumeric.py:59: RuntimeWarning: overflow encountered in accumulate
  return bound(*args, **kwds)
C:\Users\Rajae\AppData\Local\Temp\ipykernel_18036\738709997.py:19: RuntimeWarning: invalid value encountered in subtract
  drawdown = cumulative_returns - np.maximum.accumulate(cumulative_returns)


In [17]:
import torch

# Define a function for making predictions
def predict(model, data):
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        # Convert input data to torch tensor
        input_tensor = torch.tensor(data.values, dtype=torch.float32)
        
        # Ensure input tensor is 2D or 3D
        if input_tensor.dim() == 1:
            input_tensor = input_tensor.unsqueeze(0)  # Add batch dimension if input is 1D
        elif input_tensor.dim() == 2:
            input_tensor = input_tensor.unsqueeze(1)  # Add sequence length dimension if input is 2D
        
        # Forward pass to get predictions
        predictions = model(input_tensor)
        
        # Convert predictions to numpy array
        predictions = predictions.squeeze().numpy()
    
    return predictions


In [18]:
import pandas as pd

# Example of creating new data for prediction
new_data = {
    'EURUSD': [1.15, 1.16, 1.17, 1.18],  # Example EUR/USD closing prices
    'USDJPY': [110.0, 109.8, 110.2, 110.5]  # Example USD/JPY closing prices
}

# Convert to DataFrame
new_data_df = pd.DataFrame(new_data, index=pd.date_range('2023-01-01', periods=4, freq='D'))

print("New Data for Prediction:")
print(new_data_df)


New Data for Prediction:
            EURUSD  USDJPY
2023-01-01    1.15   110.0
2023-01-02    1.16   109.8
2023-01-03    1.17   110.2
2023-01-04    1.18   110.5


In [19]:
# Example usage of predict function with new data
predictions_gru_lstm = predict(gru_lstm_dqn, new_data_df)
predictions_gru = predict(gru_dqn, new_data_df)
predictions_lstm = predict(lstm_dqn, new_data_df)

# Example of printing predictions
print("GRU-LSTM Predictions:", predictions_gru_lstm)
print("GRU Predictions:", predictions_gru)
print("LSTM Predictions:", predictions_lstm)


GRU-LSTM Predictions: [2.6517427 2.6519353 2.6519318 2.6519606]
GRU Predictions: [4.981039  4.9815116 4.982565  4.9835186]
LSTM Predictions: [4.7613926 4.7613535 4.7616415 4.7618723]
